# Demo: OpenAI Large Language Model Chain of Thoughts Demo

이 데모에서는 GPT3 모델을 사용하여 자연 쿼리를 분석하고 지식 기반을 사용하여 더 많은 정보를 검색하고 질문에 답변하는 방법을 보여줍니다.  
2. Azure AI Search 활용: 첫 날 만든 Wikipedia 검색용 엔진 정보를 활용하는 방법을 살펴봅니다.

In [ ]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("../credentials.env")

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 2. AI Search를 이용하여 나의 데이터 기반으로 검색하는 방법  

RAG 모델은 정보 검색 시스템과 언어 모델을 결합하여, 주어진 질문에 대한 답변을 생성하기 위해 관련 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. 이 과정에서 Azure AI Search 검색 서비스가 핵심적인 역할을 합니다.

1. 정보 검색의 효율성: Azure AI Search는 대규모 데이터 세트에서 빠르게 관련 정보를 검색할 수 있는 기능을 제공합니다. RAG 모델은 이러한 검색 엔진을 사용하여 주어진 질문과 관련된 문서나 데이터를 효율적으로 찾아내며, 이를 바탕으로 더 정확한 답변을 생성할 수 있습니다.  

2. 정보의 정확성 및 신뢰성: Azure AI Search는 다양한 최적화 기법과 알고리즘을 사용하여 검색 결과의 정확성을 높입니다. 이는 RAG 모델이 더 신뢰할 수 있는 정보를 기반으로 답변을 생성할 수 있도록 도와줍니다.  

3. 확장성 및 가용성: Azure AI Search는 클라우드 기반 서비스로, 확장성이 뛰어나고 언제 어디서나 접근할 수 있는 장점이 있습니다. RAG 모델은 이러한 클라우드 서비스를 활용하여 다양한 규모와 범위의 데이터에 대한 검색 요구를 충족시킬 수 있습니다.  

4. 사용자 정의 및 적응성: Azure AI Search는 사용자가 직접 검색 알고리즘을 세밀하게 조정하고 최적화할 수 있도록 지원합니다. 이를 통해 RAG 모델은 특정 도메인이나 질문 유형에 맞게 검색 기능을 맞춤 설정할 수 있으며, 이는 전반적인 답변의 품질을 향상시키는 데 기여합니다.  

> 아래는 다음과 같은 사전 작업을 했다는 전제하에 진행할 수 있습니다.
- 해당 검색엔진은 OpenAI에서 제공하는 백터 정보가 포함된 Wikipeia 문서의 http://azure.studydev.com/openai/vector_sample.csv 파일을 업로드하고 색인화 했습니다.
- 백터 데이터를 Azure AI Search 엔진으로 색인화하는 방법 소개 영상: https://www.youtube.com/watch?v=XsqPjHYZD04

In [ ]:
# Azure Cognitive Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
import os

endpoint    = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
credential  = AzureKeyCredential(os.getenv('AZURE_AI_SEARCH_API_KEY'))
index_name  = os.getenv('AZURE_AI_SEARCH_INDEX')
api_version = os.getenv('AZURE_AI_SEARCH_API_VERSION')
embedding_model_name = "text-embedding-ada-002"

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential, api_version=api_version)


question     = "4월과 8월은 어떤 차이점이 있는지 알려주세요."
embedding    = client.embeddings.create(input = question, model=embedding_model_name).data[0].embedding
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="chunkVector", exhaustive=True)

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["id", "title", "chunkVector"],
)  


### Collected information for further processing

In [ ]:
grounding = ""
for result in results:  
    # print(f"Title: {result['title']}")  
    # print(f"Score: {result['@search.score']}")  
    # print(f"Text:  {result['text']}")  
    # print(f"id:    {result['id']}\n")  
    grounding = grounding + " \n " + result['chunkVector']

In [ ]:
input = question
prompt=f'''Answer the following questions as best you can. You have access to the following tools:
Question: {input}
Gronding data: {grounding}
Thought:'''


In [ ]:
print(prompt)

### Step 3: GPT3: What do I need to do to answer this question?

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [
        {"role":"system", "content":"You are a helpful  assistant. If you have data that can be expressed in a table, use Markdown to print it as a table. You can only Korean language. "},
        {"role":"user","content": prompt}
    ],
    max_tokens=600
)

print(response.choices[0].message.content)